In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils import data

from tqdm.auto import tqdm

from sklearn import model_selection as ms

In [2]:
import os
import utils as u

In [3]:
DATA_DIR = "../data/graph/"

In [4]:
os.listdir(DATA_DIR)

['out_edges_train.npy',
 'nodes_train.npy',
 'in_edges_train.npy',
 'nodes_test.npy',
 'in_edges_test.npy']

In [5]:
train_nodes = np.load(DATA_DIR + "nodes_train.npy")
train_edges = np.load(DATA_DIR + "in_edges_train.npy")

y = np.load(DATA_DIR + "out_edges_train.npy")

test_nodes = np.load(DATA_DIR + "nodes_test.npy")
test_edges = np.load(DATA_DIR + "in_edges_test.npy")

In [6]:
sub = pd.read_csv("../data/raw/sample_submission.csv")

In [7]:
def print_shapes(**kwargs):
    for key, item in dict(**kwargs).items():
        print(f"{key:^11} has shape of {str(item.shape):^10}")

In [8]:
print_shapes(train_nodes=train_nodes, train_edges=train_edges, y=y, test_nodes=test_nodes, test_edges=test_edges)

train_nodes has shape of (85003, 29, 5)
train_edges has shape of (85003, 29, 29, 6)
     y      has shape of (85003, 29, 29, 1)
test_nodes  has shape of (45772, 29, 5)
test_edges  has shape of (45772, 29, 29, 6)


In [9]:
class SimpleNet(nn.Module):
    
    def __init__(self, max_atoms, max_edges, dim_edges,
                 atom_embedding_dim=32, edge_hidden_dim=64, edge_conv_dim=32, projection_dim=32):
        super().__init__()
        
        self.max_edges = max_edges
        self.projection_dim = projection_dim
        self.edge_conv_dim = edge_conv_dim
        
        self.atom_embedding = nn.Linear(in_features=max_atoms, out_features=atom_embedding_dim)
        self.edge_hidden = nn.Linear(in_features=dim_edges, out_features=edge_hidden_dim)
        
        self.edge_convolution = nn.Conv2d(in_channels=edge_hidden_dim, out_channels=edge_conv_dim, kernel_size=5, padding=2, stride=1)
        self.edge_batchnorm = nn.BatchNorm2d(edge_conv_dim)
        
        self.out_projection = nn.Linear(
            in_features=(edge_conv_dim*max_edges)+atom_embedding_dim,
            out_features=projection_dim*max_edges
        )
        self.out_batchnorm = nn.BatchNorm2d(projection_dim)
        
        self.final_layer = nn.Linear(in_features=projection_dim, out_features=1)
        
    def forward(self, atoms, edges):
        e_atoms = F.relu(self.atom_embedding(atoms))
        
        e_edges = F.relu(self.edge_hidden(edges))
        e_edges = F.relu(self.edge_convolution(e_edges.permute(0, 3, 1, 2)))
        e_edges = self.edge_batchnorm(e_edges).permute(0, 2, 3, 1)
        
        e_edges = e_edges.reshape(-1, self.max_edges, self.max_edges*self.edge_conv_dim)
        concat = torch.cat([e_edges, e_atoms], dim=-1)
        #print(concat.size())
        concat = F.relu(self.out_projection(concat))
        #print(concat.size())
        concat = concat.reshape(-1, self.max_edges, self.max_edges, self.projection_dim)
        concat = self.out_batchnorm(concat.permute(0, 3, 1, 2)).permute(0, 2, 3, 1)
        
        return self.final_layer(concat)
        

In [10]:
valid_losses = []
train_losses = []
n_folds = 5
n_epochs = 100
random_state = 42
device = torch.device("cuda:0")
batch_size = 1024
lr = 0.001
cv = ms.KFold(n_splits=n_folds, random_state=random_state, shuffle=True)

netconfig = {
    'atom_embedding_dim': 64,
    'edge_hidden_dim': 128,
    'edge_conv_dim': 64,
    'projection_dim': 16,
    'max_atoms': 5,
    'max_edges': 29,
    'dim_edges': 6
}

In [11]:
def custom_collate_fn(samples):
    return samples[0], samples[1], samples[2]

In [12]:
split_idx = 0

for train_idx, valid_idx in cv.split(train_nodes):
    print("-"*50)
    print(f"Training for split {split_idx}")
    split_idx += 1
    train_part = data.TensorDataset(
        torch.from_numpy(train_nodes[train_idx]),
        torch.from_numpy(train_edges[train_idx]),
        torch.from_numpy(y[train_idx])
    )
    
    valid_part = data.TensorDataset(
        torch.from_numpy(train_nodes[valid_idx]),
        torch.from_numpy(train_edges[valid_idx]),
        torch.from_numpy(y[valid_idx])
    )
    
    train_loader = data.DataLoader(train_part, batch_size=batch_size, shuffle=False)
    valid_loader = data.DataLoader(valid_part, batch_size=batch_size, shuffle=False)
    
    #s, a, c = train_dataset[train_idx[:128]]
    #print(s.shape, a.shape, c.shape)
    
    model = SimpleNet(**netconfig)
    model = model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    lr_sched = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=7, threshold=0.001, verbose=True)
    
    criterion = nn.L1Loss()
    step_counter = 0
    split_train_loss = []
    split_valid_loss = []
    
    for i in range(n_epochs):
        
        train_loss = []
        valid_loss = []
        
        print(f"    Epoch {i}")
        model.train()
        for atoms, edges, y_true in tqdm(train_loader, desc='train'):
            atoms = atoms.to(device)
            edges = edges.to(device)
            y_true = y_true.to(device)
            
            predict = model(atoms, edges)
            loss = torch.log(criterion(predict, y_true))
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            
            train_loss.append(loss.item())
            
        model.eval()
        with torch.no_grad():
            for atoms, edges, y_true in tqdm(valid_loader, desc='valid'):
                atoms = atoms.to(device)
                edges = edges.to(device)
                y_true = y_true.to(device)
            
                predict = model(atoms, edges)
                loss = torch.log(criterion(predict, y_true))
                
                valid_loss.append(loss.item())
                
        lr_sched.step(np.mean(valid_loss))
                
        print(f"    Mean train mae: {np.mean(train_loss)}")
        print(f"    Mean valid mae: {np.mean(valid_loss)}")
        
        split_train_loss += train_loss
        split_valid_loss += valid_loss
        
    print(f"Mean split train loss: {np.mean(split_train_loss)}")
    print(f"Mean split valid loss: {np.mean(split_valid_loss)}")
    
    train_losses.append(np.mean(split_train_loss))
    valid_losses.append(np.mean(split_valid_loss))

--------------------------------------------------
Training for split 0
    Epoch 0



    Mean train mae: 0.9045248923461828
    Mean valid mae: 0.8620365209439221
    Epoch 1



    Mean train mae: 0.8513435043060957
    Mean valid mae: 0.8461863959536833
    Epoch 2



    Mean train mae: 0.8430956479773593
    Mean valid mae: 0.8338642532334608
    Epoch 3



    Mean train mae: 0.8298704386202257
    Mean valid mae: 0.8254986668334288
    Epoch 4


...


    Mean train mae: -1.057256437726875
    Mean valid mae: -0.9485461326206431
    Epoch 96



    Mean train mae: -1.0584141307913546
    Mean valid mae: -0.9541567528949064
    Epoch 97



    Mean train mae: -1.0599644700927076
    Mean valid mae: -0.9464633114197675
    Epoch 98



    Mean train mae: -1.0615852955283958
    Mean valid mae: -0.962739933939541
    Epoch 99



    Mean train mae: -1.0623707937271292
    Mean valid mae: -0.9415085420889013
Mean split train loss: -0.7680416597580052
Mean split valid loss: -0.5782785061201738


In [14]:
CV_train, train_std = np.mean(train_losses), np.std(train_losses)
CV_valid, valid_std = np.mean(valid_losses), np.std(valid_losses)

print(f"CV train logmae score: {CV_train:.4f}+-{train_std:.6f}")
print(f"CV valid logmae score: {CV_valid:.4f}+-{valid_std:.6f}")

CV train logmae score: -0.7692+-0.018588
CV valid logmae score: -0.6331+-0.052241
